# Трекинг мяча

Имея модель детекции футбольного мяча, попробуем реализовать алгоритм его трекинга и отрисовку траектории. Основываться будем только на результатах детекции модели.

Загрузим модель и определим нужные пути

In [1]:
import os
from glob import glob
import torch
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.interpolate import NearestNDInterpolator
from typing import List, Any

from tqdm import tqdm

In [2]:
sc_raw_ds_path = 'data/SoccerNetGS'

challenge_pics_paths = sorted(glob(f'{sc_raw_ds_path}/challenge/SNMOT-021/img1/*'))

In [3]:
best_model_path = 'yolov8n_project/tune_run_22/weights/best.pt'

In [4]:
torch.cuda.empty_cache()
model = YOLO('yolov8n_project/tune_run_22/weights/best.pt')

Реализуем класс для трекинга.

Идея основывается на том, что мяч на поле должен быть всегда один. Основная проблема - модель обучена плохо и имеет низкий recall, тоесть очень часто пропукает мяч. 

Следить будем за несколькими объектами. Для каждой точки определяем, к какому уже известному отслеживаемому объекту она ближе всего. Если расстояние до всех объектов больше N, значит это новый объект. Если информации об известном объекте нет, то он дополняется нулями на текущей итерации. Объекты, которые не обновлялись (дополнялись нулями) дольше, чем K итераций, удаляются. Отрисовывается тот объект, у которого в последних M итерациях больше всего информации (меньше всего нулей). Отрисовывается последние T итераций.

In [10]:
class Tracker():
    def __init__(self, pics_paths: List[str], model: Any, 
                 dist_threshold: int = 200, update_threshold: int = 10,
                 compare_threshold: int = 10, num_show: int = 25):
        """Init Tracker instance.
    
        Args:
            pics_paths (List[str]): path to pictures for tracking
            model (Any): model for object detection
            dist_threshold (int, optional): max distance for detection to belong to known track. Defaults to 200.
            update_threshold (int, optional): num iter with no update to delete known track. Defaults to 10.
            compare_threshold (int, optional): num iter to compare which track to draw. Defaults to 10.
            num_show (int, optional): num iter to show. Defaults to 25.
        """
        self.pics_paths = pics_paths
        self.num_frames = len(pics_paths)
        self.model = model

        self.dist_threshold = dist_threshold
        self.update_threshold = update_threshold
        self.compare_threshold = compare_threshold
        self.num_show = num_show

        self.res_video_name = 'tracking.avi'
        self.fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.reso = (1920, 1080)
        self.fps = 25

    def _detect(self, frame_num: int, conf: int = 0.1, iou: float = 0.7):
        """Perform detection on the image.

        Args:
            frame_num (int): number of current frame.
            conf (int, optional): confidence score threshold. Defaults to 0.1.
            iou (float, optional): IoU score threshold. Defaults to 0.7.
        """
        detection = self.model.predict(self.pics_paths[frame_num], conf=conf, iou=iou, verbose=False)[0]
        ann_frame = detection.plot()
        result = {
            'conf': detection.boxes.conf.cpu().numpy(),
            'xywh': detection.boxes.xywh.cpu().numpy(),
        }

        return result, ann_frame

    def track(self):
        """Track the ball"""

        video = cv2.VideoWriter(self.res_video_name, self.fourcc, self.fps, self.reso)

        # init list to store objects
        objects = []

        for i, frame in enumerate(self.pics_paths):

            # get detection for current frame
            boxes, ann_frame = self._detect(i)
            if boxes['conf'].size > 0:
                most_confident_idx = np.argmax(boxes['conf'])
                xywh = boxes['xywh'][most_confident_idx]
                coord_c = xywh[:2]

                # if there is no known tracks
                if not objects:
                    objects.append(np.zeros((self.num_frames, 2), dtype=np.int32))
                    objects[0][i] = coord_c

                # if there is, compare by dist and append, or make new
                else:
                    min_dist = self.dist_threshold
                    nearest_id = -1
                    for id, pos in enumerate(objects):
                        last_pos = np.max(np.nonzero(pos[:, 0]))
                        dist = np.linalg.norm(coord_c - pos[last_pos])
                        if dist < min_dist:
                            min_dist = dist
                            nearest_id = id

                    if min_dist == self.dist_threshold:
                        objects.append(np.zeros((self.num_frames, 2), dtype=np.int32))
                        objects[-1][i] = coord_c

                    elif nearest_id != -1:
                        objects[nearest_id][i] = coord_c

            
            # delete objects that haven't been updated
            to_delete = []
            for id, pos in enumerate(objects):
                if i >= self.update_threshold and not np.any(pos[i-self.update_threshold:i+1, :]):
                    to_delete.append(id)

            for id in to_delete:
                objects.pop(id)

            # decide which object to draw
            pts_to_draw = None
            last_n_check = i if i < self.compare_threshold else self.compare_threshold
            last_n_draw = i if i < self.num_show else self.num_show
            if len(objects) > 1:
                min_last_frames = 0
                for id, obj in enumerate(objects):
                    last_frames = obj[i-last_n_check:i+1, :]
                    nz_count = sum(np.all(last_frames, axis=1))
                    if nz_count > min_last_frames:
                        min_last_frames = nz_count
                        pts_to_draw = obj[i-last_n_draw:i+1, :]

            elif len(objects) == 1:
                pts_to_draw = objects[-1][i-last_n_draw:i+1, :]

            # draw objects
            if pts_to_draw is not None:
                # interpolate zeros for smoother trajectory
                mask = np.where(~(pts_to_draw == 0))
                interp = NearestNDInterpolator(np.transpose(mask), pts_to_draw[mask])
                pts = interp(*np.indices(pts_to_draw.shape))
                # draw poly
                pts = pts.reshape((-1, 1, 2)).astype(np.int32)
                ann_frame = cv2.polylines(ann_frame, [pts], isClosed=False, color=(255, 0, 0), thickness=10)

            video.write(ann_frame)

        cv2.destroyAllWindows()
        video.release()            


In [11]:
t = Tracker(challenge_pics_paths[300:500], model)
t.track()